# Average 3 Fold predictions

In [110]:
import json
import numpy as np
from pathlib import Path
import pandas as pd

# fold_paths = [
#     "/homes/fhoerst/histo-projects/CellViT/results/PanNuke/Revision/CellViT/Common-Loss/SAM-H/Best-Setting/2023-09-09T233134_CellViT-SAM-H-Fold-1",
#     "/homes/fhoerst/histo-projects/CellViT/results/PanNuke/Revision/CellViT/Common-Loss/SAM-H/Best-Setting/2023-09-10T014745_CellViT-SAM-H-Fold-2",
#     "/homes/fhoerst/histo-projects/CellViT/results/PanNuke/Revision/CellViT/Common-Loss/SAM-H/Best-Setting/2023-09-10T042813_CellViT-SAM-H-Fold-3"
# ]

fold_paths = [
    "/homes/fhoerst/histo-projects/CellViT/results/PanNuke/Revision/CellViTStarDist/Common-Loss/SAM-H/Unshared-decoder/Our-Setting/2023-09-15T151110_CellViTSAMStarDist-H-Unshared-Fold-1",
    "/homes/fhoerst/histo-projects/CellViT/results/PanNuke/Revision/CellViTStarDist/Common-Loss/SAM-H/Unshared-decoder/Our-Setting/2023-09-18T065152_CellViTSAMStarDist-H-Unshared-Fold-2",
    "/homes/fhoerst/histo-projects/CellViT/results/PanNuke/Revision/CellViTStarDist/Common-Loss/SAM-H/Unshared-decoder/Our-Setting/2023-09-18T065522_CellViTSAMStarDist-H-Unshared-Fold-3"
]

In [111]:
detection_results = []
cell_pq_results = []
tissue_metrics = []
global_f1 = []
global_prec = []
global_rec = []


for fold, fold_path in enumerate(fold_paths):
    json_path = Path(fold_path) / "inference_results.json"
    
    with open(json_path, 'r') as file:
        data = json.load(file)
    detection_results.append(pd.DataFrame.from_dict(data["nuclei_metrics_d"]))
    cell_pq_results.append(pd.DataFrame(data["nuclei_metrics_pq"], index=[0]))
    tissue_metrics.append(pd.DataFrame.from_dict(data["tissue_metrics"]))
    global_f1.append(float(data["dataset"]["f1_detection"]))
    global_prec.append(data["dataset"]["precision_detection"])
    global_rec.append(data["dataset"]["recall_detection"])

    
detection_avg = pd.concat([each.stack() for each in detection_results],axis=1)\
             .apply(lambda x:x.mean(),axis=1)\
             .unstack()
cell_pq_avg = pd.concat([each.stack() for each in cell_pq_results],axis=1)\
             .apply(lambda x:x.mean(),axis=1)\
             .unstack()           
tissue_metrics_avg = pd.concat([each.stack() for each in tissue_metrics],axis=1)\
             .apply(lambda x:x.mean(),axis=1)\
             .unstack()
             
global_f1 = np.mean(global_f1)
global_prec = np.mean(global_prec)
global_rec = np.mean(global_rec)

### Detection-Scores

In [112]:
detection_avg.T

,f1_cell,prec_cell,rec_cell
Neoplastic,0.701863,0.722160,0.682690
Inflammatory,0.584258,0.595634,0.573612
Connective,0.519655,0.534876,0.505431
Dead,0.384843,0.443671,0.340534
Epithelial,0.723682,0.736731,0.711335


In [113]:
print(f"f1: {global_f1}")
print(f"prec: {global_prec}")
print(f"rec: {global_rec}")

f1: 0.8182216537115305
prec: 0.8425907302775381
rec: 0.7952431762813573


### Cell-Type-PQ-Scores

In [109]:
cell_pq_avg

,Neoplastic,Inflammatory,Connective,Dead,Epithelial
0,0.508306,0.374857,0.283907,0.095283,0.469167


### Tissue-Type-PQ-Scores

In [26]:
tissue_metrics_avg.T

,Dice,Jaccard,mPQ,bPQ
adrenal_gland,0.852938,0.759609,0.495020,0.697319
bile-duct,0.798428,0.741551,0.472102,0.630390
bladder,0.804538,0.759970,0.575600,0.654418
breast,0.835017,0.736184,0.508851,0.655268
cervix,0.736887,0.737079,0.489327,0.592687
colon,0.725100,0.679941,0.424513,0.513301
esophagus,0.847420,0.751006,0.537335,0.658277
headneck,0.692956,0.673157,0.490089,0.549478
kidney,0.803840,0.742512,0.540934,0.652299
liver,0.868471,0.776027,0.506541,0.712221


In [24]:
row_mean = tissue_metrics_avg.mean(axis=1)
row_mean

Dice       0.815431
Jaccard    0.738738
mPQ        0.484597
bPQ        0.643721
dtype: float64

In [25]:
row_mean = tissue_metrics_avg.std(axis=1)
row_mean

Dice       0.049415
Jaccard    0.028870
mPQ        0.050317
bPQ        0.048988
dtype: float64